<h1>Simulating continous portfolio optimization</h1>

In [1]:
from datetime import datetime, timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
#stocks = ['AAPL', 'AMZN', 'MSFT', 'TSLA']
stocks = ['AAPL', 'TSLA']

In [3]:
# Download data
data_df = yf.download(stocks, start='2018-01-01')

# Convert the index to datetime
data_df.index = pd.to_datetime(data_df.index)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


In [4]:
display(data_df)

Price            Close                    High                     Low  \
Ticker            AAPL        TSLA        AAPL        TSLA        AAPL   
Date                                                                     
2018-01-02   40.479843   21.368668   40.489245   21.474001   39.774865   
2018-01-03   40.472782   21.150000   41.017967   21.683332   40.409337   
2018-01-04   40.660782   20.974667   40.764179   21.236668   40.437540   
2018-01-05   41.123711   21.105333   41.210657   21.149332   40.665476   
2018-01-08   40.970974   22.427334   41.267063   22.468000   40.872274   
...                ...         ...         ...         ...         ...   
2025-04-07  181.460007  233.289993  194.149994  252.000000  174.619995   
2025-04-08  172.419998  221.860001  190.339996  250.440002  169.210007   
2025-04-09  198.850006  272.200012  200.610001  274.690002  171.889999   
2025-04-10  190.419998  252.399994  194.779999  262.489990  183.000000   
2025-04-11  192.774994  247.509995  193.699799  257.739990  186.059998   

Price                         Open                 Volume             
Ticker            TSLA        AAPL        TSLA       AAPL       TSLA  
Date                                                                  
2018-01-02   20.733334   39.986361   20.799999  102223600   65283000  
2018-01-03   21.036667   40.543281   21.400000  118071600   67822500  
2018-01-04   20.378668   40.545634   20.858000   89738400  149194500  
2018-01-05   20.799999   40.757123   21.108000   94640000   68868000  
2018-01-08   21.033333   40.970974   21.066668   82271200  147891000  
...                ...         ...         ...        ...        ...  
2025-04-07  214.250000  177.199997  223.779999  160466300  183453800  
2025-04-08  217.800003  186.699997  245.000000  120859500  171603500  
2025-04-09  223.880005  171.949997  224.690002  184395900  219433400  
2025-04-10  239.330002  189.070007  260.000000  121685000  180875500  
2025-04-11  244.071198  186.179993  251.839996   21144943   43371729  

[1830 rows x 10 columns]

In [5]:
def get_data_before(data_df: pd.DataFrame, end: str):
    filtered_df = data_df[data_df.index <= end]
    return filtered_df

In [6]:
def optimize_portfolio(data_df, stocks: list, end: str):
    current_data_df = get_data_before(data_df, end)
    
    # Calculating daily returns
    current_data_df = current_data_df['Close']
    x = current_data_df.pct_change()

    # Storing the weights, returns and Sharpe ratios for each portfolio
    p_weights, p_returns, p_risk, p_sharpe = [], [], [], []

    # Running a for loop, generate the random weights and calculate the returns, volatility and Sharpe ratio of the portfolio.
    count = 5000
    for k in range(0, count):
        # Randomly assign a weight to each stock in our portfolio, and then calculate the metrics for that portfolio, including the Sharpe ratio.
        wts = np.random.uniform(size = len(stocks))
        wts = wts/np.sum(wts)
        p_weights.append(wts)
    
        # Returns
        mean_ret = (x.mean() * wts).sum()*252
        p_returns.append(mean_ret)
        
        # Volatility
        ret = (x * wts).sum(axis = 1)
        annual_std = np.std(ret) * np.sqrt(252)
        p_risk.append(annual_std)
            
        # Sharpe ratio
        sharpe = (np.mean(ret) / np.std(ret))*np.sqrt(252)
        p_sharpe.append(sharpe)

    # Finding the optimal index
    max_ind = np.argmax(p_sharpe)

    # Finding the max sharpe ratio
    max_sharpe_ratio = p_sharpe[max_ind]
    
    # Finding the optimal stock weights
    optimal_stock_weights = np.round(p_weights[max_ind], 2)

    return round(float(max_sharpe_ratio), 2), optimal_stock_weights.tolist()

In [7]:
max_sharpe_ratio, optimal_stock_weights = optimize_portfolio(
    data_df=data_df,
    stocks=stocks, 
    end='2025-04-01'
)

# Max Sharpe ratio
print("The maximum sharpe ratio: ", max_sharpe_ratio)
# Stocks
print("The stocks: ", stocks)
# Stock Weights
print("The optimal stock weights that gives the maximum sharpe ratio: ", optimal_stock_weights)

The maximum sharpe ratio:  1.06
The stocks:  ['AAPL', 'TSLA']
The optimal stock weights that gives the maximum sharpe ratio:  [0.72, 0.28]


In [8]:
first_max_sharpe_ratio = max_sharpe_ratio
first_optimal_stock_weights = optimal_stock_weights
allowed_difference = 0.05

for i in range(10):
    print(f"Test Run nr.", i+1)
    
    difference_found = False
    
    test_max_sharpe_ratio, test_optimal_stock_weights = optimize_portfolio(
        data_df=data_df,
        stocks=stocks, 
        end='2025-04-01'
    )
    
    for test_index in range(len(stocks)):
        test_value = first_optimal_stock_weights[test_index]
        if abs(test_optimal_stock_weights[test_index] - test_value) > allowed_difference:
            difference_found = True
            print(f"Share difference for {stocks[test_index]} is bigger than {allowed_difference} for the same optimization run")
            print(f"First Optimal Stock Weight = {test_value}, and Current Optimal Stock Weight = {test_optimal_stock_weights[test_index]}")

    if not difference_found:
        print("No big difference was found.")
        
    print("----------------------------------")

print("Testing is done.")

Test Run nr. 1
No big difference was found.
----------------------------------
Test Run nr. 2
No big difference was found.
----------------------------------
Test Run nr. 3
No big difference was found.
----------------------------------
Test Run nr. 4
No big difference was found.
----------------------------------
Test Run nr. 5
No big difference was found.
----------------------------------
Test Run nr. 6
No big difference was found.
----------------------------------
Test Run nr. 7
No big difference was found.
----------------------------------
Test Run nr. 8
No big difference was found.
----------------------------------
Test Run nr. 9
No big difference was found.
----------------------------------
Test Run nr. 10
No big difference was found.
----------------------------------
Testing is done.


In [9]:
today = "2025-04-01"
max_sharpe_ratio, optimal_stock_weights_array = optimize_portfolio(
    data_df=data_df,
    stocks=stocks, 
    end=today
)

optimal_stock_weights = {
    "max_sharpe_ratio": max_sharpe_ratio,
    "optimal_stock_weights": {}
}

for stock_index in range(len(stocks)):
    optimal_stock_weights["optimal_stock_weights"][stocks[stock_index]] = optimal_stock_weights_array[stock_index]

optimal_stock_weights_dict = optimal_stock_weights["optimal_stock_weights"]
print("Optimization date: ", today)
print("The maximum sharpe ratio: ", max_sharpe_ratio)
print("The optimal stock weights that gives the maximum sharpe ratio: \n", optimal_stock_weights_dict)
print("Sum of weights: ", round(sum(optimal_stock_weights_array)))

Optimization date:  2025-04-01
The maximum sharpe ratio:  1.06
The optimal stock weights that gives the maximum sharpe ratio: 
 {'AAPL': 0.72, 'TSLA': 0.28}
Sum of weights:  1


In [10]:
def get_previous_non_empty(df, date):
    current_date = pd.to_datetime(date)
    while current_date >= df.index.min():
        # Filter dataframe for the given date
        filtered_df = df[df.index == current_date]
        
        if not filtered_df.empty:
            return filtered_df
        else:
            # If empty, go to the previous date
            current_date -= pd.Timedelta(days=1)
    
    # Return None if no valid data is found
    return None

def get_prices_at_date(
    data_df: pd.DataFrame, 
    date: str, 
    price_type: str
) -> dict[tuple, float]:
    
    filtered_df = get_previous_non_empty(data_df, date)

    # Initialize an empty dictionary
    result_dict = {}
    # Loop over the columns
    for col in filtered_df.columns:
        if price_type in col:
            price, ticker = col  # Extract Price and Ticker from the MultiIndex
            if ticker not in result_dict:
                result_dict[ticker] = {}  # Initialize nested dictionary for each Ticker
            result_dict[ticker][price] = filtered_df[col].tolist()  # Assign the corresponding values to the dictionary
            
    return result_dict

In [11]:
def buy(
    data_df: pd.DataFrame, 
    date: str, 
    optimal_stock_weights: dict[str, float],
    balance: float
) -> (dict[str, float] | None, float | None, float | None):
    
    price_type = "Open"
    
    current_prices = get_prices_at_date(
        data_df=data_df, 
        date=date, 
        price_type=price_type
    )

    current_shareholding_value = 0
    input_balance = balance
    current_shareholding = {}

    min_balance_required = 0
    for stock, price in current_prices.items():
        min_balance_required += price[price_type][0]
    if input_balance < min_balance_required:
        return None, None, balance
    
    for stock, price in current_prices.items():
        optimized_stock_weight = optimal_stock_weights[stock]
        buy_total_amount = input_balance * optimized_stock_weight
        if buy_total_amount > balance or balance == 0:
            return None, None, balance
        else:
            buy_stock_price = price[price_type][0]
            current_shareholding[stock] = round( buy_total_amount / buy_stock_price, 2)
            current_shareholding_value += buy_total_amount
            balance -= buy_total_amount

    return current_shareholding, round(current_shareholding_value, 2), round(balance, 2)

In [12]:
balance = 1000.00 # in Dollars
shareholding = {} # in Shares
shareholding_value = 0 # in Dollars

In [13]:
current_shareholding, current_shareholding_value, current_balance = buy(
    data_df=data_df, 
    date=today, 
    optimal_stock_weights=optimal_stock_weights_dict, 
    balance=balance
)

shareholding = current_shareholding if current_shareholding is not None else shareholding
shareholding_value = current_shareholding_value if current_shareholding_value is not None else shareholding_value
balance = current_balance

print(f"{balance = }") # Dollars
print(f"{shareholding = }") # Shares
print(f"{shareholding_value = }") # Dollars

balance = 0.0
shareholding = {'AAPL': 3.28, 'TSLA': 1.06}
shareholding_value = 1000.0


In [14]:
def sell(
    data_df: pd.DataFrame, 
    date: str, 
    shareholding: dict[str, float],
    shareholding_value: float,
    balance: int
) -> (dict[str, float], float, float):

    price_type = "Close"
    
    current_prices = get_prices_at_date(
        data_df=data_df, 
        date=date, 
        price_type=price_type
    )

    current_shareholding = shareholding
    current_shareholding_value = shareholding_value
    current_balance = balance
    for stock, share in shareholding.items():
        price = current_prices[stock][price_type][0]
        sell_amount = round(price * share, 2)
        current_balance += sell_amount
        current_shareholding[stock] = 0

    all_stocks_are_sold = True
    for _, share in shareholding.items():
        if share != 0:
            all_stocks_are_sold = False

    if all_stocks_are_sold:
        current_shareholding = {}
        current_shareholding_value = 0

    return current_shareholding, round(current_shareholding_value, 2), round(current_balance, 2)

In [15]:
current_shareholding, current_shareholding_value, current_balance = sell(
    data_df=data_df, 
    date=today, 
    shareholding=shareholding,
    shareholding_value=shareholding_value,
    balance=balance
)

balance = current_balance
shareholding = current_shareholding
shareholding_value = current_shareholding_value

print(f"{balance = }") # Dollars
print(f"{shareholding = }") # Shares
print(f"{shareholding_value = }") # Dollars

balance = 1016.63
shareholding = {}
shareholding_value = 0


In [16]:
# Start and end dates
start_date = datetime(2025, 4, 1)
end_date = datetime(2025, 4, 7)

# Assets at investment start
balance = 1000.00 # in Dollars
shareholding = {} # in Shares
shareholding_value = 0 # in Dollars

# Stores optimal weights and its evaluation metrics
simulation_optimal_weights = {}

# Loop through each date
today = start_date
while today <= end_date:

    one_day = timedelta(days=1)
    yesterday = today - one_day
    yesterday_str = yesterday.strftime('%Y-%m-%d')
    today_str = today.strftime('%Y-%m-%d')
    tomorrow = today + one_day
    tomorrow_str = tomorrow.strftime('%Y-%m-%d')

    max_sharpe_ratio, optimal_stock_weights_array = optimize_portfolio(
        data_df=data_df,
        stocks=stocks, 
        end=yesterday_str
    )
    simulation_optimal_weights[yesterday_str] = {
        "max_sharpe_ratio": max_sharpe_ratio,
        "optimal_stock_weights": {}
    }
    for stock_index in range(len(stocks)):
        simulation_optimal_weights[yesterday_str]["optimal_stock_weights"][stocks[stock_index]] = optimal_stock_weights_array[stock_index]
    optimal_stock_weights = simulation_optimal_weights[yesterday_str]["optimal_stock_weights"]
    print("Optimization date: ", today_str)
    print("The maximum sharpe ratio: ", max_sharpe_ratio)
    print("The optimal stock weights that gives the maximum sharpe ratio: \n", optimal_stock_weights)
    print("Sum of weights: ", round(sum(optimal_stock_weights_array)))

    current_shareholding, current_shareholding_value, current_balance = buy(
        data_df=data_df, 
        date=today_str, 
        optimal_stock_weights=optimal_stock_weights, 
        balance=balance
    )
    shareholding = current_shareholding if current_shareholding is not None else shareholding
    shareholding_value = current_shareholding_value if current_shareholding_value is not None else shareholding_value
    balance = current_balance

    current_shareholding, current_shareholding_value, current_balance = sell(
        data_df=data_df, 
        date=today_str, 
        shareholding=shareholding,
        shareholding_value=shareholding_value,
        balance=balance
    )
    balance = current_balance
    shareholding = current_shareholding
    shareholding_value = current_shareholding_value

    print("Current Balance ($): "f"{balance = }")
    print("Current Shareholding (%): ", shareholding)
    print("Current Value of Shareholding ($): ", shareholding_value)
    print("------------------------------------------")
    
    today += timedelta(days=1)

Optimization date:  2025-04-01
The maximum sharpe ratio:  1.05
The optimal stock weights that gives the maximum sharpe ratio: 
 {'AAPL': 0.72, 'TSLA': 0.28}
Sum of weights:  1
Current Balance ($): balance = 1016.63
Current Shareholding (%):  {}
Current Value of Shareholding ($):  0
------------------------------------------
Optimization date:  2025-04-02
The maximum sharpe ratio:  1.06
The optimal stock weights that gives the maximum sharpe ratio: 
 {'AAPL': 0.72, 'TSLA': 0.28}
Sum of weights:  1
Current Balance ($): balance = 1057.77
Current Shareholding (%):  {}
Current Value of Shareholding ($):  0
------------------------------------------
Optimization date:  2025-04-03
The maximum sharpe ratio:  1.06
The optimal stock weights that gives the maximum sharpe ratio: 
 {'AAPL': 0.71, 'TSLA': 0.29}
Sum of weights:  1
Current Balance ($): balance = 1051.68
Current Shareholding (%):  {}
Current Value of Shareholding ($):  0
------------------------------------------
Optimization date:  20